In [2]:
#import the working library
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from scipy.stats import zscore
%matplotlib inline

In [3]:
#read tickers as specified in excel in addition to market 
all_tickers = pd.read_excel('mapping.xlsx')[['Ticker', 'Market']]
#list all tickers
all_tickers

,Ticker,Market
0,ARGT,Emerging
1,EWA,Developed
2,EWO,Developed
3,EWK,Developed
4,EWZ,Emerging
5,EWC,Developed
6,ECH,Emerging
7,MCHI,Emerging
8,ICOL,Emerging
9,EDEN,Developed


In [4]:
factor_description = pd.read_excel('factor description.xlsx', index_col='Short Name')
factor_description

,Name,Category
Short Name,,
mom_21,Short-term Absolute Momentum (21-trading day),Technical
mom_63,Medium-term Absolute Momentum (63-trading day),Technical
mom_252,Long-term Absolute Momentum (252-trading day),Technical
mr_21,Percentage over Short-term Moving Average (21-...,Technical
mr_63,Percentage over Medium-term Moving Average (63...,Technical
mr_252,Percentage over Long-term Moving Average (252-...,Technical
div_yield,Dividend Yield,Fundamental
pb,Price/Book Ratio,Fundamental
pe,Price/Earning Ratio,Fundamental


In [5]:
#specified factor and date
start_date_train = '2016-01-01'
end_date_train = '2018-12-31'
start_date_test = '2019-01-01'
end_date_test = '2019-12-31'

# Technical Factors

In [6]:
factors = ['mom_252', 'mom_63', 'mom_21', 
           'mr_252', 'mr_63', 'mr_21']

In [7]:
for factor_to_analyze in ['mom_252']:
    description_factor_to_analyze = factor_description.loc[factor_to_analyze, 'Name']
    print(description_factor_to_analyze)

    #list all etf in datafrome
    all_etf_combined_df = pd.DataFrame()

    #going throw each etf forward return 
    for file in os.listdir('forward return'):
        fr_df = pd.read_csv('forward return/'+file, index_col='Date')
        fr_df['Ticker'] = file[:-4]
        try:
            factor_df = pd.read_csv('factor data/'+factor_to_analyze+'/'+file[:-4]+'.csv', index_col='Date')
            combined_df = fr_df.join(factor_df).rename({factor_to_analyze: 'Factor'}, axis=1)
            combined_df['Factor'] = combined_df['Factor'].fillna(method='ffill')
            all_etf_combined_df = pd.concat([all_etf_combined_df, combined_df.dropna().reset_index()], axis=0, ignore_index=True)
        except:
            continue

    all_etf_combined_df = all_etf_combined_df.join(all_tickers.set_index('Ticker'), on='Ticker').dropna()
    all_etf_combined_df = all_etf_combined_df.sort_values(['Date', 'Market', 'Ticker']).set_index(['Date', 'Market', 'Ticker'])[start_date_train:end_date_train]
    all_etf_combined_df['Winsorized Factor'] = np.nan
    all_etf_combined_df['Normalized Winsorized Factor'] = np.nan
    all_etf_combined_df['Normalized Winsorized Factor Quartile'] = -1

    #combine all factors
    for date in all_etf_combined_df.index.get_level_values(0).unique():
        for market in ['Developed', 'Emerging']:
            n_etf = all_etf_combined_df.loc[date].loc[market].shape[0]
            all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'] = winsorize(all_etf_combined_df.loc[date].loc[market]['Factor'], limits=1/n_etf*2)
            all_etf_combined_df.loc[date].loc[market]['Normalized Winsorized Factor'] = zscore(all_etf_combined_df.loc[date].loc[market]['Winsorized Factor'])
            all_etf_combined_df.loc[date].loc[market]['Normalized Winsorized Factor Quartile'] = pd.qcut(all_etf_combined_df.loc[date].loc[market]['Normalized Winsorized Factor'], 5, labels=False)
    for 

Long-term Absolute Momentum (252-trading day)


In [8]:
all_etf_combined_df

f_rtn    Factor  Winsorized Factor  \
Date       Market    Ticker                                          
2016-01-04 Developed EDEN   -0.046028  0.185859           0.128830   
                     EFNL   -0.043960  0.018566           0.018566   
                     EIRL   -0.063144  0.214778           0.128830   
                     EIS    -0.061430  0.071966           0.071966   
                     ENOR   -0.053008 -0.162176          -0.162176   
...                               ...       ...                ...   
2018-12-31 Emerging  MCHI    0.127220 -0.196151          -0.196151   
                     QAT     0.052043  0.211525          -0.021842   
                     THD     0.097688 -0.075031          -0.075031   
                     TUR     0.176247 -0.413340          -0.309426   
                     UAE     0.038578 -0.143227          -0.143227   

                             Normalized Winsorized Factor  \
Date       Market    Ticker                                 
2016-01-04 Developed EDEN                        1.611045   
                     EFNL                        0.414749   
                     EIRL                        1.611045   
                     EIS                         0.994108   
                     ENOR                       -1.546177   
...                                                   ...   
2018-12-31 Emerging  MCHI                       -0.580370   
                     QAT                         1.357642   
                     THD                         0.766270   
                     TUR                        -1.839786   
                     UAE                         0.008051   

                             Normalized Winsorized Factor Quartile  
Date       Market    Ticker                                         
2016-01-04 Developed EDEN                                        4  
                     EFNL                                        3  
                     EIRL                                        4  
                     EIS                                         4  
                     ENOR                                        0  
...                                                            ...  
2018-12-31 Emerging  MCHI                                        1  
                     QAT                                         4  
                     THD                                         3  
                     TUR                                         0  
                     UAE                                         2  

[34506 rows x 5 columns]